# 실험해 봐야될것. 

1. 데이터 집단이 1~6 으로 나뉘어져있다. 이것을 각기 다르게 train dataset 으로 넣어서 학습을 시켜보자 . 
2. 데이터에 대한 좀 더 정확한 분석. 강우량, 강수량, 유역별 강수량. 이것들을 어떻게 해결할지 고민해 보자. 
3. train/test data 는 1 ~ 10 / 11 ~ 25 이렇게 나눠서 진행해 보는 것으로 하자. 

  

https://dschloe.github.io/python/python_edu/07_deeplearning/deep_learning_lstm/

를 참고하여 일단 작성중


In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from os.path import join

In [2]:
df = pd.read_excel('rain_data_drop_scaled_datetime.xlsx')
df.head()

,홍수사상번호,date,유입량,데이터집단 1,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,...,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,avg_A_waterfall,avg_B_waterfall,avg_C_waterfall,avg_D_waterfall,avg_D_waterlevel,avg_E_waterlevel
0,1,2006-07-10 07:00:00,0.008630,0.019488,0.021277,0.019074,0.027668,0.030534,0.093930,0.153560,...,0.032389,0.030651,0.093930,0.155313,0.020875,0.019462,0.030201,0.030908,0.155043,0.155043
1,1,2006-07-10 08:00:00,0.009926,0.019184,0.021277,0.021798,0.027668,0.030534,0.093291,0.153312,...,0.040486,0.038314,0.093291,0.154916,0.020875,0.022243,0.034228,0.034771,0.154919,0.154919
2,1,2006-07-10 09:00:00,0.011529,0.019488,0.021277,0.024523,0.027668,0.030534,0.093291,0.153064,...,0.040486,0.042146,0.093291,0.154518,0.020875,0.025023,0.034899,0.036703,0.154795,0.154795
3,1,2006-07-10 10:00:00,0.013919,0.022229,0.021277,0.027248,0.027668,0.030534,0.093291,0.153064,...,0.060729,0.053640,0.093291,0.154320,0.023857,0.027804,0.046980,0.043786,0.154719,0.154719
4,1,2006-07-10 11:00:00,0.017731,0.024970,0.021277,0.032698,0.031621,0.038168,0.093291,0.153064,...,0.072874,0.061303,0.093291,0.153923,0.030318,0.033364,0.055034,0.050869,0.154507,0.154507


In [3]:
feature_col = ['데이터집단 1', 'Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 10','Unnamed: 11','Unnamed: 12','데이터집단 2','Unnamed: 14','Unnamed: 15','Unnamed: 16','Unnamed: 17','Unnamed: 18','Unnamed: 19','데이터집단 3','Unnamed: 21','Unnamed: 22','Unnamed: 23','Unnamed: 24','Unnamed: 25','Unnamed: 26','데이터 집단 4','Unnamed: 28','Unnamed: 29','Unnamed: 30','Unnamed: 31','Unnamed: 32','Unnamed: 33','데이터 집단 5','Unnamed: 35','Unnamed: 36','Unnamed: 37','Unnamed: 38','Unnamed: 39','Unnamed: 40','데이터 집단 6','Unnamed: 42','Unnamed: 43','Unnamed: 44','Unnamed: 45','Unnamed: 46','Unnamed: 47','avg_A_waterfall', 'avg_B_waterfall', 'avg_C_waterfall', 'avg_D_waterfall', 'avg_D_waterlevel', 'avg_E_waterlevel']
label_col = ['유입량']

1. 데이터셋 (train data, validation data
2. LSTM model 
3. compile
4. fit
5. 결과 보기 



In [4]:
len(df)

3051

In [5]:
def make_dataset(data, label, window_size=6):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)

시계열 데이터의 데이터셋은 보통 window_size 라고 정의한다. 

- window_size 는 과거 기간의 주가 데이터에 기반하여 다음날의 종가를 예측할 것인가를 정하는 parameter 이다. 

- 만약 과거 20일을 기반으로 내일 데이터를 예측한다고 가정하면, window_size = 20 이 되는 것이다. 

- 실제 100일의 과거 데이터를 기반으로 데이터셋을 분리하도록 한다. 

- 이때, make_dataset 이라는 함수를 만든다. feature_list & label_list 를 분리 한다. 

- test_size = 200 은 학습은 과거로부터 200일 이전의 데이터를 학습하게 되되t , test 를 위해서 이후 100일의 데이터로 모델이 주가를 예측하도록 한 다음, 실제 데이터와 오차가 얼마나 있는지 확인해 보도록 하겠다고 함. 



# 잠시만, 얘는지금 test, validaiton data 에 대한 개념을 나눠서 생각하고 있는 것이겠지? 

## train_feature 에는 유입량을 제외한 train 시켜야되는 데이터를 포함. 
## train_label 에는 유입량이 들어가게 되는 것이다. 

In [6]:
TEST_SIZE = 160
train = df[:-TEST_SIZE]
test = df[-TEST_SIZE:]

train_feature = train[feature_col]
train_label = train[label_col]

In [7]:
train_feature

,데이터집단 1,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,데이터집단 2,Unnamed: 14,Unnamed: 15,...,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,avg_A_waterfall,avg_B_waterfall,avg_C_waterfall,avg_D_waterfall,avg_D_waterlevel,avg_E_waterlevel
0,0.019488,0.021277,0.019074,0.027668,0.030534,0.093930,0.153560,0.019184,0.020772,0.019074,...,0.032389,0.030651,0.093930,0.155313,0.020875,0.019462,0.030201,0.030908,0.155043,0.155043
1,0.019184,0.021277,0.021798,0.027668,0.030534,0.093291,0.153312,0.019488,0.020772,0.021798,...,0.040486,0.038314,0.093291,0.154916,0.020875,0.022243,0.034228,0.034771,0.154919,0.154919
2,0.019488,0.021277,0.024523,0.027668,0.030534,0.093291,0.153064,0.022229,0.020772,0.024523,...,0.040486,0.042146,0.093291,0.154518,0.020875,0.025023,0.034899,0.036703,0.154795,0.154795
3,0.022229,0.021277,0.027248,0.027668,0.030534,0.093291,0.153064,0.024970,0.020772,0.027248,...,0.060729,0.053640,0.093291,0.154320,0.023857,0.027804,0.046980,0.043786,0.154719,0.154719
4,0.024970,0.021277,0.032698,0.031621,0.038168,0.093291,0.153064,0.034409,0.026706,0.032698,...,0.072874,0.061303,0.093291,0.153923,0.030318,0.033364,0.055034,0.050869,0.154507,0.154507
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2886,0.069378,0.018237,0.000000,0.003953,0.003817,0.125240,0.733069,0.025148,0.017804,0.000000,...,0.004049,0.003831,0.125240,0.732075,0.017893,0.000000,0.004027,0.003863,0.732884,0.732884
2887,0.025148,0.006079,0.000000,0.003953,0.003817,0.123323,0.733565,0.012512,0.005935,0.000000,...,0.004049,0.003831,0.123323,0.732671,0.005964,0.000000,0.004027,0.003863,0.733290,0.733290
2888,0.012512,0.003040,0.000000,0.003953,0.003817,0.122045,0.733813,0.010309,0.002967,0.000000,...,0.004049,0.003831,0.122045,0.733069,0.002982,0.000000,0.004027,0.003863,0.733622,0.733622
2889,0.010309,0.003040,0.000000,0.003953,0.003817,0.120767,0.734061,0.010000,0.002967,0.000000,...,0.004049,0.003831,0.120767,0.733466,0.002982,0.000000,0.004027,0.003863,0.734009,0.734009


In [8]:
train_label
len(train_label)

2891

In [9]:
# train dataset
train_feature, train_label = make_dataset(train_feature, train_label, 20)

In [10]:
print(train_feature)
print(len(train_feature))

[[[0.01948843 0.0212766  0.01907357 ... 0.03090792 0.15504297 0.15504297]
  [0.01918392 0.0212766  0.02179837 ... 0.03477141 0.15491893 0.15491893]
  [0.01948843 0.0212766  0.02452316 ... 0.03670316 0.15479488 0.15479488]
  ...
  [0.17356882 0.12462006 0.16621253 ... 0.23245332 0.15296667 0.15296667]
  [0.1772229  0.12462006 0.16893733 ... 0.23438506 0.15393783 0.15393783]
  [0.18087698 0.13069909 0.17166213 ... 0.23889247 0.15519928 0.15519928]]

 [[0.01918392 0.0212766  0.02179837 ... 0.03477141 0.15491893 0.15491893]
  [0.01948843 0.0212766  0.02452316 ... 0.03670316 0.15479488 0.15479488]
  [0.02222899 0.0212766  0.02724796 ... 0.04378622 0.15471886 0.15471886]
  ...
  [0.1772229  0.12462006 0.16893733 ... 0.23438506 0.15393783 0.15393783]
  [0.18087698 0.13069909 0.17166213 ... 0.23889247 0.15519928 0.15519928]
  [0.18392205 0.1337386  0.17438692 ... 0.24146813 0.15661915 0.15661915]]

 [[0.01948843 0.0212766  0.02452316 ... 0.03670316 0.15479488 0.15479488]
  [0.02222899 0.021276

## 얼라리요 값이 반올림되는게 있네? 이렇게 되도 되려나? 

In [11]:
print(train_label)
print(len(train_label))

[[0.06010701]
 [0.05826407]
 [0.05869976]
 ...
 [0.022716  ]
 [0.02221129]
 [0.0217162 ]]
2871


0.06010701 이라는 것은 22열에 있는 것이다. 앞에 20개를 뭔가로 설정하고 
21 번째 데이터가 train_label 로 설정이 되어있는 상태임. 

0.0217162 는 2892 열에 있는 데이터. (테스트 데이터 빈칸 바로 전까지 임)

22~2892 에 해당하는 2870개의 데이터로 분리가 된 상황.

원래 train data 는 2890 개인데, window_size = 20 으로 인해서 이렇게 된거 같음. 

## train_test_split 
얘는 정확하게 말하자면, train 과 validation 을 나누는 것이다.

train/test 으로 구분이 되어있었던 데이터 셋을

train data 를 train/ validation 으로 나누는 작업을 한다고 한다. 

그리고 다들 아는 것처럼 validation 을 통해서 학습한 모델을 평가하는 역할을 하게 되는것이다. 

In [17]:
# train, validation set 생성, 
# test_data size =30% 로 일단 측정해 두었다. 
# validation 에 30%, train dp 70% 를 맥인 상황. 
# x_train.shape 가 1818,20,48 로 되어있음. 내 생각에는 가운데에 있는 값은,,, 아마도 윈도우 사이즈 를 의미하는 것 같고, 
#48 의 경우에는 feature 갯수가 48개라서 그런거 같음


from sklearn.model_selection import train_test_split 


x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=633)
print("x_train.shape", "x_valid.shape")
print(x_train.shape, x_valid.shape)
#(1981, 20, 48) (850, 20, 48)

test_feature = test[feature_col]
test_label = test[label_col]



# test dataset (실제 예측 해볼 데이터)
test_feature, test_label = make_dataset(test_feature, test_label, 20)
print("test_feature.shape, test_label.shape")
print(test_feature.shape, test_label.shape)
# ((180, 20, 4), (180, 1))

x_train.shape x_valid.shape
(2238, 20, 48) (633, 20, 48)
test_feature.shape, test_label.shape
(140, 20, 48) (140, 1)


# LSTM model 

In [18]:
train_feature.shape

(2871, 20, 48)

In [19]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM
import tensorflow as tf


model = Sequential()
model.add(LSTM(50,input_shape=(train_feature.shape[1], train_feature.shape[2]),activation='relu',return_sequences=False))
model.add(Dense(1))
model.compile(loss = 'mean_squared_error', optimizer='adam', metrics = ['accuracy'])

# early_stop = EarlyStopping(monitor='val_loss', patience=5)
# filename = os.path.join('tmp_checkpoint.h5')
# checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50)                19800     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 19,851
Trainable params: 19,851
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
history = model.fit(x_train, y_train, 
                    epochs=200, 
                    batch_size=20,
                    verbose = 2,
                    validation_data=(x_valid, y_valid)
#                     callbacks=[early_stop, checkpoint])                   
                   )



Epoch 1/200
112/112 - 3s - loss: 0.0082 - accuracy: 8.9366e-04 - val_loss: 0.0021 - val_accuracy: 0.0000e+00
Epoch 2/200
112/112 - 1s - loss: 0.0016 - accuracy: 8.9366e-04 - val_loss: 0.0012 - val_accuracy: 0.0000e+00
Epoch 3/200
112/112 - 1s - loss: 9.4659e-04 - accuracy: 8.9366e-04 - val_loss: 8.1975e-04 - val_accuracy: 0.0000e+00
Epoch 4/200
112/112 - 1s - loss: 7.9676e-04 - accuracy: 8.9366e-04 - val_loss: 6.8401e-04 - val_accuracy: 0.0000e+00
Epoch 5/200
112/112 - 1s - loss: 6.2942e-04 - accuracy: 8.9366e-04 - val_loss: 6.5563e-04 - val_accuracy: 0.0000e+00
Epoch 6/200
112/112 - 1s - loss: 6.6362e-04 - accuracy: 8.9366e-04 - val_loss: 6.0113e-04 - val_accuracy: 0.0000e+00
Epoch 7/200
112/112 - 1s - loss: 5.5943e-04 - accuracy: 8.9366e-04 - val_loss: 6.5109e-04 - val_accuracy: 0.0000e+00
Epoch 8/200
112/112 - 1s - loss: 4.7328e-04 - accuracy: 8.9366e-04 - val_loss: 5.5267e-04 - val_accuracy: 0.0000e+00
Epoch 9/200
112/112 - 1s - loss: 4.5880e-04 - accuracy: 8.9366e-04 - val_loss: 4

KeyboardInterrupt: 